In [1]:
import json
import os
import re
import xml.etree.ElementTree as et
import cv2
import math
from sklearn.model_selection import GridSearchCV, train_test_split
from skimage import feature as ft
from sklearn import svm
import numpy as np
path="your_dir/VOC3000/Annotations/3000_"
model_path1=r'your_dir/vgg1.h5'
model_path2=r'your_dir/vgg2.h5'
imgpath="your_dir/VOC3000/JPEGImages/3000_"
num=3147
def get_namespace(element):  # 获取节点的namespace
    m = re.match('\{.*\}', element.tag)
    return m.group(0) if m else ''
data_set=[]
monum=0
mrnum=0
map_obj={}
map_rel={}
for i in range(num):
    if i % 10 ==0:
        print(i)
    strnum=str(i).zfill(6)
    tpath=path+strnum+".xml"
    if not os.path.exists(tpath):
        continue
    tree=et.parse(tpath)
    
    root=tree.getroot()
    ns=get_namespace(root)
    #print(ns)
    #print(root.tag)
    img_data=[]
    obj=[]
    relation_dict=[]
    img_size=[]
    for child in root:
        #print(child.tag)
        
        if child.tag=="object":
            tmp=[]
            for cd in child:
                if cd.tag=="name":
                    #print(cd.text)
                    if cd.text not in map_obj.keys():
                        if cd.text.find("_")==-1:
                            map_obj[cd.text]=monum
                            monum+=1
                    tmp.append(cd.text)
                if cd.tag=="object_id":
                    tmp.append(cd.text)
                if cd.tag=="bndbox":
                    for n in cd:
                        tmp.append(n.text)
            #print(tmp)
            obj.append(tmp)
            continue
        if child.tag=="size":
            for cd in child:
                img_size.append(cd.text)
    obj_data=[]
    if len(obj)==3:
        flag=1
        relnum=0
        for h in range(len(obj)):
            if obj[h][0].find("_")!=-1:
                relnum+=1
                flag=0
                tmp=obj[h][0].split("_")
                a1=0
                a2=len(tmp)-1
                #这一步是因为数据集奇葩，15th person lying in beach object里面没有beach，吐了
                if tmp[a1] in map_obj.keys():
                    relation_dict.append(tmp[a1])
                else:
                    flag=1
                rela=tmp[a1+1]
                for o in range(a1+2,a2):
                    rela=rela+"_"+tmp[o]
                if rela not in map_rel.keys():
                    map_rel[rela]=mrnum
                    mrnum+=1
                relation_dict.append(rela)
                if tmp[a2] in map_obj.keys():
                    relation_dict.append(tmp[a2])
                else:
                    flag=1
            else:
                obj_data.append(obj[h])
        if flag==1 or relnum>1:
            continue
        img_data.append(obj_data)
        img_data.append(relation_dict)
        img_data.append(img_size)
        img_data.append(i)
        data_set.append(img_data)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [2]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers

class vgg16:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0001
        self.x_shape = [64,64,3]

        self.model = self.build_model()
        

    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model

    def normalize(self,X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    def train(self,x_train,x_test,y_train,y_test,num):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)


        #model=self.model
        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        self.model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = self.model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)
        self.model.save_weights('vgg'+str(num)+'.h5')
        return self.model
def vgg_train(x_train,x_test,y_train,y_test,num):
    model=vgg16()
    model.train(x_train,x_test,y_train,y_test,num)


def get_pix_gray(img_num,loc):
    img=cv2.imread(imgpath+str(img_num).zfill(6)+".jpg",cv2.IMREAD_GRAYSCALE).astype(np.float32)
    xmin=int(loc[0])
    ymin=int(loc[1])
    xmax=math.ceil(loc[2])
    ymax=math.ceil(loc[3])
    
    #print(img.shape)
    #print(img)
    return img[xmin:xmax,ymin:ymax]
def get_pix_col(img_num,loc):
    img=cv2.imread(imgpath+str(img_num).zfill(6)+".jpg").astype(np.float32)
    xmin=int(loc[0])
    ymin=int(loc[1])
    xmax=math.ceil(loc[2])
    ymax=math.ceil(loc[3])
    #print(img.shape)
    #print(img)
    img=img[ymin:ymax,xmin:xmax,:]
    #img=img[xmin:xmax,ymin:ymax,:]
    #cv2.imshow("img",img)
    #cv2.imwrite("img"+str(img_num)+".jpg",img)
    #img=cv2.resize(img,(64,64))
    return img

len= 819
0
person nextto car 1
1
person sitting_on chair 4
2
person riding bicycle 18
3
person riding horse 19
4
person nextto car 29
5
person drinking bottle 31
6
person jumping_on horse 35
7
person riding horse 38
8
cat lying_on sofa 39
9
person riding bicycle 44
10
person nextto horse 46
11
person riding horse 47
12
person drinking bottle 48
13
person riding horse 49
14
bicycle nextto car 50
15
person jumping_on horse 51
16
person jumping_on horse 53
17
person riding bicycle 58
18
bicycle nextto car 59
19
person jumping_on horse 68
20
person sitting_on chair 70
21
person nextto horse 71
22
person drinking bottle 79
23
person nextto horse 82
24
person sitting_on chair 92
25
person drinking bottle 93
26
person lying_on sofa 94
27
person nextto horse 103
28
person jumping_on horse 108
29
person riding horse 111
30
person nextto car 112
31
person riding horse 120
32
cat lying_on sofa 121
33
person lying_on sofa 122
34
person nextto car 131
35
person sitting_on chair 133
36
person drinki

286
person sitting_on chair 1092
287
person sitting_on chair 1098
288
person nextto bicycle 1102
289
dog lying_on sofa 1118
290
person nextto horse 1119
291
person riding bicycle 1125
292
person sitting_on chair 1130
293
person sitting_on sofa 1132
294
person riding bicycle 1136
295
bicycle nextto car 1137
296
person nextto bicycle 1141
297
person riding horse 1146
298
person riding horse 1147
299
person nextto horse 1152
300
person riding horse 1163
301
person riding bicycle 1164
302
person sitting_on sofa 1165
303
dog lying_on sofa 1166
304
person riding horse 1168
305
person nextto car 1170
306
person sitting_on sofa 1171
307
person jumping_on horse 1173
308
person riding horse 1175
309
cat lying_on sofa 1177
310
person nextto horse 1179
311
person riding horse 1185
312
cat lying_on sofa 1189
313
bicycle nextto car 1190
314
person riding horse 1192
315
person drinking bottle 1193
316
person nextto bicycle 1195
317
person jumping_on horse 1197
318
person sitting_on chair 1198
319
cat

561
person sitting_on chair 2102
562
person jumping_on horse 2116
563
person sitting_on chair 2117
564
person sitting_on sofa 2122
565
person nextto bicycle 2123
566
cat lying_on sofa 2124
567
cat lying_on sofa 2125
568
person nextto bicycle 2126
569
bicycle nextto car 2127
570
person sitting_on chair 2129
571
person riding bicycle 2131
572
bicycle nextto car 2134
573
person jumping_on horse 2141
574
person sitting_on chair 2144
575
person riding horse 2145
576
person drinking bottle 2154
577
dog lying_on sofa 2155
578
bicycle nextto car 2156
579
person nextto horse 2158
580
person drinking bottle 2160
581
person sitting_on chair 2167
582
person nextto car 2172
583
person drinking bottle 2176
584
person nextto bicycle 2185
585
person nextto horse 2186
586
person jumping_on horse 2189
587
person riding horse 2192
588
person jumping_on horse 2198
589
person riding bicycle 2200
590
person sitting_on sofa 2201
591
dog lying_on sofa 2205
592
person nextto car 2208
593
bicycle nextto car 220

In [3]:
train_s=[]
train_o=[]
train_y=[]
#print("len=",len(data_set))
for i in range(len(data_set)):
    print(i)
    Subject=data_set[i][1][0]
    Object=data_set[i][1][2]
    Relation=data_set[i][1][1]
    imgnum=data_set[i][3]
    #print(Subject,Relation,Object,imgnum)
    subnum=map_obj[Subject]
    objnum=map_obj[Object]
    relnum=map_rel[Relation]
    img_size=data_set[i][2]
    idx=0
    if data_set[i][0][0][0]==Subject:
        idx=0
    else:
        idx=1
    sub_loc=data_set[i][0][idx][1:]
    obj_loc=data_set[i][0][1-idx][1:]
    for j in range(len(img_size)):
        img_size[j]=float(img_size[j])
    for j in range(len(sub_loc)):
        sub_loc[j]=float(sub_loc[j])
    for j in range(len(obj_loc)):
        obj_loc[j]=float(obj_loc[j])
    imgs=get_pix_col(imgnum,sub_loc)
    imgo=get_pix_col(imgnum,obj_loc)
    imgs=np.array(imgs)
    imgo=np.array(imgo)
    s=cv2.resize(imgs,(64,64))
    o=cv2.resize(imgo,(64,64))
    train_s.append(s)
    train_o.append(o)
    train_y.append(relnum)
xs_train,xs_test,ys_train,ys_test=train_test_split(train_s,train_y,test_size=0.2)
xo_train,xo_test,yo_train,yo_test=train_test_split(train_o,train_y,test_size=0.2)
vgg_train(xs_train,xs_test,ys_train,ys_test,1):
vgg_train(xo_train,xo_test,yo_train,yo_test,2):

Using TensorFlow backend.


In [ ]:
import matplotlib
import os
from keras.models import load_model
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image
from pathlib import Path
import cv2
def normalize(x_train,x_test):
    x_test=np.array(x_test,dtype=np.float32)
    x_train=np.array(x_train,dtype=np.float32)
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    X_train = (x_train-mean)/(std+1e-7)
    X_test = (x_test-mean)/(std+1e-7)
    return X_test

if Path(model_path1).exists():
    print("路径正确")
tmp=vgg16()
my_smodel=tmp.model
my_smodel.load_weights(model_path1,by_name=False)
my_smodel.summary()
tmp2=vgg16()
my_omodel=tmp2.model
my_omodel.load_weights(model_path2,by_name=False)
my_omodel.summary()
#my_smodel=load_model(model_path)
cnt=0
for i in range(len(xs_test)):
    img=xs_test[i]
    cv2.imwrite("tmp.jpg",img)
    img=image.load_img("tmp.jpg",target_size=(64,64))
    img=image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=preprocess_input(img)
    p=my_smodel.predict(img)
    p=np.argmax(p,axis=1)[0]
    
    img2=xo_test[i]
    cv2.imwrite("tmp2.jpg",img2)
    img2=image.load_img("tmp2.jpg",target_size=(64,64))
    img2=image.img_to_array(img2)
    img2=np.expand_dims(img2,axis=0)
    img2=preprocess_input(img2)
    p2=my_omodel.predict(img2)
    p2=np.argmax(p2,axis=1)[0]
    #print(p)
    if p==ys_test[i] and p2==yo_test[i]:
        cnt+=1
    print(p,ys_test[i])
    #print(img.shape)
print(cnt/len(xs_test))